In [37]:
import pandas as pd

pre_links_update_df = pd.read_csv('final_companies_data.csv')
temp_updated_links_df = pd.read_csv('temp_updated_links.csv')

pre_links_update_df.drop('Rank', axis=1, inplace=True)
page_ranks_dict = {}
for i, r in pd.read_csv('d.csv').iterrows():
    pg_rank = 0
    try:
        pg_rank = float(str(r['pg_rank']))
    except:
        pg_rank = 0
    page_ranks_dict[str(r['domain'])] = pg_rank

legit_companies = set(temp_updated_links_df['name'].unique())
rows_to_delete = []

for i, r in pre_links_update_df.iterrows():
    if r['name'] not in legit_companies:
        rows_to_delete.append(i)
pre_links_update_df = pre_links_update_df.drop(rows_to_delete)

temp_updated_links_df = temp_updated_links_df[['name', 'students_careers_url', 'website']]
pre_links_update_df = pre_links_update_df.drop(['students_careers_url', 'website'], axis=1)
print(pre_links_update_df.shape, temp_updated_links_df.shape)
final_df = pd.merge(pre_links_update_df, temp_updated_links_df, on='name')
list_of_scores = []
max_num_employees = 0
max_revenue = 0
for i, r in final_df.iterrows():
    if len(r['num_employees']) > 1:
        r['num_employees'] = r['num_employees'].replace(',', '')
        r['num_employees'] = r['num_employees'].strip()
        if r['num_employees'] == '-':
            continue
        max_num_employees = max(max_num_employees, int(r['num_employees']))
    if len(r['Revenue(in millions)']) > 1:
        r['Revenue(in millions)'] = r['Revenue(in millions)'].replace(',', '')
        r['Revenue(in millions)'] = r['Revenue(in millions)'].strip()
        if r['Revenue(in millions)'] == '-':
            continue
        max_revenue = max(max_revenue, float(r['Revenue(in millions)']))
for i, r in final_df.iterrows():
    score = 0
    try:
        score = 4 * page_ranks_dict[r['website']]
    except:
        print(r['name'])
    if len(r['num_employees']) > 1:
        r['num_employees'] = r['num_employees'].replace(',', '')
        r['num_employees'] = r['num_employees'].strip()
        if r['num_employees'] is not '-':
            score += 3 * int(r['num_employees']) / max_num_employees
    if len(r['Revenue(in millions)']) > 1:
        r['Revenue(in millions)'] = r['Revenue(in millions)'].replace(',', '')
        r['Revenue(in millions)'] = r['Revenue(in millions)'].strip()
        if r['Revenue(in millions)'] is not '-':
            score += 3 * float(r['Revenue(in millions)']) / max_revenue
    list_of_scores.append(score)
final_df['score'] = list_of_scores
final_df = final_df.sort_values(by='score', ascending=False)
final_df.to_csv('final_df.csv', index=False)

(899, 8) (899, 3)
CA Wily Technology
Tacit Software
Netcordia, Inc.
